In [1]:
%load_ext autoreload
%autoreload 2

from crossref.restful import Works, Journals
from artfinder import CrossrefArticle, Crossref
import pandas as pd
import datetime
import re
import logging

logging.basicConfig(level=logging.INFO)

In [2]:
crosref = Crossref(app='artfinder', email='aapopov1@mephi.ru')
df = crosref.query(author='Kabashin').get_df()

INFO:crossref.restful:GET request to https://api.crossref.org/works, data={'query.author': 'Kabashin', 'cursor': '*', 'rows': 100}, headers={'user-agent': 'artfinder/undefined (undefined; mailto:aapopov1@mephi.ru) BasedOn: CrossrefAPI/1.6.0'}


INFO:crossref.restful:Sleep for 0.02 seconds
INFO:crossref.restful:GET request to https://api.crossref.org/works, data={'query.author': 'Kabashin', 'cursor': 'DnF1ZXJ5VGhlbkZldGNoBgAAAAAHgD-BFi10RWc3ay1xUzB1LXA4djZWbEdpQ1EAAAAAE2qDIBZWX2tkV3YyWVFVcV9hOFZHWGN4cURnAAAAADMkE28WTUpsaGN0RGFRbS1yN0ZYWTJ3MG5pUQAAAAAuCk8BFmZPZjNDVWhLUnh1RDFtUWM4WFIzUHcAAAAAHzP1VhZwMUUtd3VRM1RzU3RxcXpMUVg3MFZBAAAAABEgWuEWZjkwZGpsbmpTUG10QXVtRFhYcXlaUQ==', 'rows': 100}, headers={'user-agent': 'artfinder/undefined (undefined; mailto:aapopov1@mephi.ru) BasedOn: CrossrefAPI/1.6.0'}
INFO:crossref.restful:Sleep for 0.02 seconds
INFO:crossref.restful:GET request to https://api.crossref.org/works, data={'query.author': 'Kabashin', 'cursor': 'DnF1ZXJ5VGhlbkZldGNoBgAAAAAHgD-BFi10RWc3ay1xUzB1LXA4djZWbEdpQ1EAAAAAE2qDIBZWX2tkV3YyWVFVcV9hOFZHWGN4cURnAAAAADMkE28WTUpsaGN0RGFRbS1yN0ZYWTJ3MG5pUQAAAAAuCk8BFmZPZjNDVWhLUnh1RDFtUWM4WFIzUHcAAAAAHzP1VhZwMUUtd3VRM1RzU3RxcXpMUVg3MFZBAAAAABEgWuEWZjkwZGpsbmpTUG10QXVtRFhYcXlaUQ==', 'rows':

In [16]:
df.columns

Index(['publisher', 'license', 'is_referenced_by_count', 'link', 'authors',
       'abstract', 'title', 'doi', 'type', 'journal', 'issn', 'volume',
       'issue', 'start_page', 'end_page', 'publication_date', 'references'],
      dtype='object')

In [128]:
kaba = pd.read_excel('database/processed/kabashin.xlsx')

In [129]:
kaba['ablation'] = kaba['ablation'].astype(bool)
kaba['title'] = kaba['title'].str.lower()

In [142]:
full_df = df.join(kaba, on='doi')

ValueError: You are trying to merge on object and int64 columns for key 'doi'. If you wish to proceed you should use pd.concat

Select articles about laser ablation

In [131]:
kaba_ablation = kaba[kaba['ablation'] == True]
kaba_ablation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 151 entries, 1 to 250
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     151 non-null    object
 1   abstract  55 non-null     object
 2   doi       151 non-null    object
 3   ablation  151 non-null    bool  
dtypes: bool(1), object(3)
memory usage: 4.9+ KB


Get statistics of words from titles

In [132]:
kaba_ablation[kaba_ablation['ablation']]['title'].map(lambda x: re.split(r'\W+', x)).explode().value_counts().head(20)

title
of               138
laser            109
nanoparticles    105
for               69
and               57
in                49
ablation          48
by                39
gold              35
synthesized       34
synthesis         31
applications      29
femtosecond       27
based             24
plasmonic         19
aqueous           19
si                18
therapy           18
with              17
on                17
Name: count, dtype: int64

Check titles of articles where word *nanoparticles* is missing

In [141]:
no_nps = kaba_ablation[~kaba_ablation['title'].str.contains(r'nanoparticles?|nanostructures?|laser|nanoclusters?|nanospheres?')]
no_nps

,title,abstract,doi,ablation
103,transforming nuclear medicine with nanoradioph...,NaN,10.1021/acsnano.1c10550,True
137,novel advanced nanotechnologies for nuclear me...,Nuclear nanomedicine forms a new research fiel...,10.1088/1742-6596/2058/1/012035,True
146,advanced electrocatalysts on the basis of bare...,NaN,10.1021/acscatal.5b01478,True
155,x-ray contrast properties of bismuth-based nan...,NaN,10.3103/s1068335623220141,True
191,graphene–gold metasurface architectures for ul...,NaN,10.1002/adma.201501754,True
215,manipulating the optically active defect–defec...,NaN,10.1021/acscatal.3c03884,True


In [115]:
all_words = kaba_ablation['title'].map(lambda x: re.split(r'\W+', x)).explode().unique()
all_words = set(all_words)
common = all_words.copy()
for title in kaba_ablation['title'].map(lambda x: re.split(r'\W+', x)):
    common = common.intersection(set(title))
common

set()

In [40]:
work = crosref.doi('10.1117/12.459737')

In [41]:
work['title']

['&lt;title&gt;Air optical breakdown on silicon as a novel method to fabricate photoluminescent Si-based nanostructures&lt;/title&gt;']